In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline
import os
import shutil
import pandas as pd
import torch.nn.functional as F

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = torchvision.datasets.MNIST(root = './data',
                                     download = True,
                                     train = True,
                                     transform = transforms.ToTensor())
trainloader =  torch.utils.data.DataLoader(dataset, batch_size = 50, shuffle = True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = torchvision.datasets.MNIST(root = './data',
                                     download = True,
                                     train = True,
                                     transform = transforms.ToTensor())
trainloader =  torch.utils.data.DataLoader(dataset, batch_size = 50, shuffle = True)



In [3]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 10),
            nn.ReLU())
        
        self.decoder = nn.Sequential(
            nn.Linear(10, 32),
            nn.ReLU(),
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.Linear(128, 784),
            nn.Sigmoid())
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [4]:
model = AutoEncoder().to(device)
critierion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-4)
for epoch in range(51):
    running_loss = 0.0
    for data in trainloader:
        inputs = data[0].to(device)
        optimizer.zero_grad()
        outputs = model(inputs.view(-1, 28*28))
        outputs = outputs.view(-1, 1, 28, 28)
        loss = critierion(inputs, outputs)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    cost = running_loss / len(trainloader)
    print("[%d] loss : %.3f" % (epoch, cost))



[0] loss : 0.085
[1] loss : 0.062
[2] loss : 0.053
[3] loss : 0.043
[4] loss : 0.038
[5] loss : 0.035
[6] loss : 0.033
[7] loss : 0.032
[8] loss : 0.031
[9] loss : 0.030
[10] loss : 0.029
[11] loss : 0.028
[12] loss : 0.028
[13] loss : 0.027
[14] loss : 0.027
[15] loss : 0.026
[16] loss : 0.026
[17] loss : 0.026
[18] loss : 0.025
[19] loss : 0.025
[20] loss : 0.025
[21] loss : 0.025
[22] loss : 0.025
[23] loss : 0.025
[24] loss : 0.024
[25] loss : 0.024
[26] loss : 0.024
[27] loss : 0.024
[28] loss : 0.024
[29] loss : 0.024
[30] loss : 0.024
[31] loss : 0.024
[32] loss : 0.023
[33] loss : 0.023
[34] loss : 0.023
[35] loss : 0.023
[36] loss : 0.023
[37] loss : 0.023
[38] loss : 0.023
[39] loss : 0.023
[40] loss : 0.023
[41] loss : 0.023
[42] loss : 0.023
[43] loss : 0.023
[44] loss : 0.023
[45] loss : 0.022
[46] loss : 0.022
[47] loss : 0.022
[48] loss : 0.022
[49] loss : 0.022
[50] loss : 0.022


In [9]:
for epoch in range(101):
    runnning_loss = 0.0
    for data in trainloader:
        inputs = data[0].to(device)
        optimizer.zero_grad()
        dirty_inputs = inputs + torch.normal(0, 0.5, size = inputs.size()).to(device)
        outputs = model(dirty_inputs.view(-1, 28*28))
        outputs = outputs.view(-1,1,28,28)
        loss = critierion(inputs, outputs)
        loss.backward()
        optimizer.step()
        runnning_loss  += loss.item()
    
    cost = runnning_loss / len(trainloader)
    print('Epoch : {%d} loss : {%.4f}' %(epoch, cost))


Epoch : {0} loss : {0.0312}
Epoch : {1} loss : {0.0292}
Epoch : {2} loss : {0.0286}
Epoch : {3} loss : {0.0283}
Epoch : {4} loss : {0.0280}
Epoch : {5} loss : {0.0278}
Epoch : {6} loss : {0.0276}
Epoch : {7} loss : {0.0274}
Epoch : {8} loss : {0.0273}
Epoch : {9} loss : {0.0271}
Epoch : {10} loss : {0.0270}
Epoch : {11} loss : {0.0268}
Epoch : {12} loss : {0.0267}
Epoch : {13} loss : {0.0265}
Epoch : {14} loss : {0.0264}
Epoch : {15} loss : {0.0263}
Epoch : {16} loss : {0.0262}
Epoch : {17} loss : {0.0261}
Epoch : {18} loss : {0.0259}
Epoch : {19} loss : {0.0259}
Epoch : {20} loss : {0.0258}
Epoch : {21} loss : {0.0257}
Epoch : {22} loss : {0.0256}
Epoch : {23} loss : {0.0256}
Epoch : {24} loss : {0.0255}
Epoch : {25} loss : {0.0254}
Epoch : {26} loss : {0.0254}
Epoch : {27} loss : {0.0253}
Epoch : {28} loss : {0.0253}
Epoch : {29} loss : {0.0252}
Epoch : {30} loss : {0.0251}
Epoch : {31} loss : {0.0251}
Epoch : {32} loss : {0.0251}
Epoch : {33} loss : {0.0250}
Epoch : {34} loss : {0.0

### Convolutional Autoencoder

In [ ]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)

class Deflatten(nn.Module):
    def __init__(self, k):
        super(Deflatten, self).__init__()
        self.k = k
    
    def forward(self, x):
        s = x.size()
        feature_size = int((s[1]//self.k)**.5)
        return x.view(s[0], self.k, feature_size, feature_size)

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        k = 16
        self.encoder = nn.Sequential(
            nn.Conv2d(1, k, 3, stride = 2), nn.ReLU(),
            nn.Conv2d(k, 2*k, 3, stride = 2), nn.ReLU(),
            nn.Conv2d(2*k, 4*k, 3, stride = 1), nn.ReLU(),
            Flatten(), nn.Linear(1024, 10), nn.ReLU())
        
        self.decoder = nn.Sequential(
            nn.Linear(10, 1024), nn.ReLU(),
            Deflatten(4*k),
            nn.ConvTranspose2d(4*k, 2*k, 3, stride = 1), nn.ReLU(),
            nn.ConvTranspose2d(2*k, k, 3, stride = 2), nn.ReLU(),
            nn.ConvTranspose2d(k, 1, 3, stride = 2, output_padding =1), nn.Sigmoid())
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

model = autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
